<a href="https://colab.research.google.com/github/monikaspec641/detr/blob/main/yolov12_object_detection_on_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content/datasets


In [ ]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.222 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 40.4/112.6 GB disk)


In [ ]:
# Git clone method (for development)

# %cd {HOME}
# !git clone github.com/ultralytics/ultralytics
# %cd {HOME}/ultralytics
# !pip install -qe ultralytics

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

## CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://v8docs.ultralytics.com/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

## Inference with Pre-trained COCO Model

### 💻 CLI

`yolo mode=predict` runs YOLOv8 inference on a variety of sources, downloading models automatically from the latest YOLOv8 release, and saving results to `runs/predict`.

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model=yolo12s.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

/content/datasets
Ultralytics 8.3.222 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12s summary (fused): 159 layers, 9,261,840 parameters, 0 gradients, 21.4 GFLOPs

image 1/1 /content/datasets/dog.jpeg: 640x384 1 person, 1 car, 1 dog, 1 handbag, 59.5ms
Speed: 2.1ms preprocess, 59.5ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 384)
Results saved to /content/datasets/runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


## Custom Training

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet
from roboflow import Roboflow
rf = Roboflow(api_key="2AUlNBrTrF7eobbzZiYc")
project = rf.workspace("pothole-detection-z7r02").project("foreignud-7pzmg")
version = project.version(8)
dataset = version.download("yolov11")


/content/datasets/datasets
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ForeignUD-8 in yolov11:: 100%|██████████| 4659/4659 [00:01<00:00, 2933.04it/s]


In [ ]:
%cd {HOME}

# Load a custom model
model = YOLO('yolo11n.pt')  # or YOLO('yolov8n.yaml')

# Train the model
results = model.train(data='/content/datasets/ForeignUD-8/data.yaml', epochs=100, imgsz=640, plots=True)

/content/datasets
Ultralytics 8.3.222 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/ForeignUD-8/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, persp

In [ ]:
%cd /content/runs/detect/train/weights/
!ls

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# After the training cell has been run, access the results object
# Note: The exact structure of the results object may vary slightly depending on the YOLO version
# You might need to inspect the results object after training to find the exact location of the metrics

if results and hasattr(results, 'metrics'):
    metrics = results.metrics  # Access metrics from the results object
    print("Per-class metrics:")
    for metric_name, metric_value in metrics.items():
        print(f"{metric_name}: {metric_value}")
elif results:
     # If metrics are not directly in results.metrics, try accessing them from the validation results
    if hasattr(results, 'results_dict'):
        print("Validation metrics:")
        for metric_name, metric_value in results.results_dict.items():
             print(f"{metric_name}: {metric_value}")
    else:
        print("Could not find metrics in the results object. Please inspect the results object after training.")
else:
    print("Could not retrieve results object after training.")